In [ ]:
import torch
from modelscope import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
torch.backends.cuda.enable_mem_efficient_sdp(False)
#torch.backends.cuda.enable_flash_sdp(False)
model_name = "deepseek-ai/deepseek-llm-7b-chat"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id



In [ ]:
import pandas as pd
df_news = pd.read_json('/home/userroot/dev/jiucai/crawl/sina_news/data/bak.stock_news.jsonl',lines=True)
batch_size = 1
line_size = df_news.shape[0]
cnt = line_size // batch_size 
for i in range(cnt):
    batch_news = df_news[i*batch_size:(i+1)*batch_size]
    batch_prompts = []
    for i, news in batch_news.iterrows():
        #print(len(news['content']))
        news = news['content'][0:4000]

        messages = [
            

            {"role": "user", "content": f"从下面内容中总结内容\n{news}"},
            {"role":"system", "content":f"你是一名专业的投研分析人员，你的任务是进行投研分析并总结，然后发布评论，如果无法总结，请回答不知道。总结内容不要超过150字,不允许分段。评论内容在总结内容之后，以【评论】开头"},

        ]
        batch_prompts.append(messages)
    #     print(len(prompt_templated))
    #     print(tokenizer.decode(input_tensor[0]))
    #     print(input_tensor.shape)
    # continue

    #prompt_templated = tokenizer.apply_chat_template(batch_prompts, tokenize=False, add_generation_prompt=True, return_tensors="pt",truncation='only_first')

    input_tensor = tokenizer.apply_chat_template(batch_prompts, tokenize=True, add_generation_prompt=True, return_tensors="pt",truncation='only_first', padding=True)

    outputs = model.generate(input_tensor.to(model.device), max_new_tokens=300, temperature = 0.0000000001)

    for i in range(batch_size):

        result = tokenizer.decode(outputs[i][input_tensor.shape[1]:], skip_special_tokens=True)
        print(f"idx {i}: {result}")
    #break

In [4]:
from vllm import LLM
import torch
from modelscope import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import os
os.environ['VLLM_USE_MODELSCOPE'] = "True"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)
model_name = "deepseek-ai/deepseek-llm-7b-chat"
tokenizer = AutoTokenizer.from_pretrained(model_name)
llm = LLM(model_name, trust_remote_code=True,dtype=torch.float16,tensor_parallel_size=2)


2024-04-07 13:47:32,771 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-04-07 13:47:32,774 - modelscope - INFO - Loading ast index from /home/userroot/.cache/modelscope/ast_indexer
2024-04-07 13:47:32,821 - modelscope - INFO - Loading done! Current index file version is 1.13.3, with md5 821edacd9c7695756b5e5583d4438f12 and a total number of 972 components indexed
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


WARNING 04-07 13:47:34 config.py:767] Casting torch.bfloat16 to torch.float16.


2024-04-07 13:47:37,111	INFO worker.py:1752 -- Started a local Ray instance.


INFO 04-07 13:47:38 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='/home/userroot/.cache/modelscope/hub/deepseek-ai/deepseek-llm-7b-chat', tokenizer='/home/userroot/.cache/modelscope/hub/deepseek-ai/deepseek-llm-7b-chat', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir='/home/userroot/.cache/modelscope/hub/deepseek-ai/deepseek-llm-7b-chat', load_format=auto, tensor_parallel_size=2, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 04-07 13:47:45 selector.py:40] Cannot use FlashAttention backend for Volta and Turing GPUs.
INFO 04-07 13:47:45 selector.py:25] Using XFormers backend.
(RayWorkerVllm pid=501925) INFO 04-07 13:47:47 selector.py:40] Cannot use FlashAttention backend for Volta and Turing GPUs.
(RayWorkerVllm pid=501925) INFO 04-07 13:47:47 selector.py:25] Using XFormers backend.
INFO 04-07 13:47:47 pynccl_utils.py:45] vLLM is using nccl==2.18.1
(RayWorkerVllm pid=501925) INFO 04-07 13:47:47 pynccl_utils.py:45] vLLM is using nccl==2.18.1
INFO 04-07 13:48:01 model_runner.py:104] Loading model weights took 6.4663 GB
(RayWorkerVllm pid=501925) INFO 04-07 13:48:06 model_runner.py:104] Loading model weights took 6.4663 GB
INFO 04-07 13:48:08 ray_gpu_executor.py:240] # GPU blocks: 3262, # CPU blocks: 1092
INFO 04-07 13:48:13 model_runner.py:791] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' 

In [8]:
from vllm import LLM, SamplingParams
import pandas as pd
df_news = pd.read_json('/home/userroot/dev/jiucai/crawl/sina_news/data/bak.stock_news.jsonl',lines=True)
df_news = df_news[0:20]
for i , news in df_news.iterrows():
    news = news['content'][0:4000]
    messages = [
                

                {"role": "user", "content": f"从下面内容中总结内容\n{news}"},
                {"role":"system", "content":f"你是一名专业的投研分析人员，你的任务是进行投研分析,先总结，然后发布评论，如果无法总结，请回答不知道。评论内容在总结内容之后，以【评论】开头。全部内容不要超过150字,不允许分段。"},

            ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, return_tensors="pt",truncation='only_first')
    sampling_params = SamplingParams(temperature=0, top_p=0.95, max_tokens=300)
    print(llm.generate(prompt,sampling_params)[0].outputs[0].text)

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.88s/it]


总结：

1. 截至2023年末，工行、农行、建行私行客户数均超过20万户，中行、招行的客户数也均超过了10万户。
2. 除招行之外，工行、中行、农行、建行的管理规模均超过2万亿元。
3. 各银行私人银行业务经营数据表明，私行直营中心建设正在加快，“人-家-企-社”是私行业务发展的重要思路。

评论：私人银行业务发展迅速，各银行在私行直营中心建设、综合营销和直接经营等方面加大力度，以满足客户需求。


Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.35s/it]


总结：

贵州茅台发布了2023年年报，公司营业总收入和归母净利润同比双双增长。第四季度营业总收入和归母净利润也同比双双增长。各项财务指标表现尚佳，毛利率、净利率、每股收益等指标均有所增长。资产质量方面，公司现金资产健康，存货周转天数较慢，需要进一步观察是否是行业原因。营收分析方面，公司现金流为正，经营性现金流和利润不匹配，但问题不大。公司盈利能力优秀，且稳健，业务体量和利润有过中速增长。分析师普遍预期2024年业绩在876.38亿元，每股收益均值在69.76元。

评论：

贵州茅台的财报显示公司业绩表现良好，各项财务指标增长明显。分析师普遍预期公司未来业绩将继续增长，但增速可能会有所减缓。基金经理对贵州茅台的持仓较为积极，易方达基金的陈皓是备受关注的基金经理之一。投资者可以关注贵州茅台的未来发展趋势，但需注意投资风险。


...

总结：君乐宝乳业集团股份有限公司正式启动IPO进程，计划在2025年完成上市，并立下500亿元的销售目标。作为中国第五大乳企，君乐宝业务范围包括婴幼儿奶粉、酸奶、低温鲜奶、常温液态奶等板块，建有25个生产工厂、25个现代化大型牧场。尽管君乐宝营收增速颇为亮眼，但资产负债率居高不下，为77.63%，远高于同行。此外，产品质量和售后服务问题频发，消费者投诉较多。

评论：君乐宝在IPO进程中面临资产负债率高企、产品质量和服务问题等挑战，如何提升品牌信任度、消除消费者疑虑，将是其未来发展的关键。


Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.49s/it]


总结：
中国发展高层论坛2024年年会于3月24日在北京举行，伊利集团高级执行总裁张剑秋在“碳中和与全球气候治理”分论坛中分享了伊利引领中国奶业绿色发展的经验。伊利集团坚持以产业协同为路径，以科技创新为引领，以数字转型为动力，积极稳妥推进碳达峰碳中和，努力实现应对气候变化和推进行业高质量发展的互促共赢。为推动实现合作伙伴的低碳转型，伊利打造了行业首个“零碳联盟”，并搭建了供应商碳排放数据收集系统，2027年，伊利将实现核心供应商100%低碳转型。

评论：
伊利集团在推动绿色发展方面表现出色，通过产业协同、科技创新和数字转型等路径，积极推进碳达峰碳中和，努力实现应对气候变化和推进行业高质量发展的互促共赢。此外，伊利还打造了行业首个“零碳联盟”，并搭建了供应商碳排放数据收集系统，以推动实现合作伙伴的低碳转型。这一举措不仅有助于伊利自身实现低碳发展，也为整个行业树立了绿色发展的典范。


Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.34s/it]

总结：

在“两山理论”“双碳战略”指引下，中国企业特别是上市公司和国企央企，正成为建设美丽中国，实现绿色低碳、可持续发展的重要行动主体。企业社会责任理念已被联合国、非政府组织、西方发达国家及普通大众广为熟知并普遍接受，各国公司尤其是跨国企业纷纷将社会责任纳入发展战略和长远规划中。我国也高度重视企业社会责任建设。ESG不仅仅是投资策略，还是一种新的价值理念和评价工具，正在重新定义企业增长的方式。改善ESG表现，对企业发展至关重要，有助于实现长期、强劲的财务业绩，实现社会价值和商业价值的同步增长。

评论：

企业践行社会责任，是实现绿色低碳、可持续发展的重要途径。在ESG理念的引领下，企业应紧密结合国家大政方针和施政所向，在应对气候变化、推进绿色发展、坚持生态文明、助力乡村振兴等领域挖掘和传播优秀企业社会责任故事。同时，政府职能部门应通过政策引导，展现企业创造的社会价值，为企业可持续发展营造良好环境。


In [6]:
import torch
import torch
import os
torch.cuda.device_count()
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#torch.empty(2, device='cuda')
torch.cuda.get_device_capability()

(7, 5)